In [4]:
import nibabel as nib
import numpy as np
from functions import search_circles, find_lines, process_central_lines, process_b_spline_central_lines, visualize_lines
from skimage import morphology, measure
import open3d as o3d

# 文件路径
#nifti_file = 'Cshape/5475-4_Canal.nii' 
nifti_file = 'qulv/5530-3/5530-3ori_canal.nii'
#nifti_file = 'all_c_shape/canal-before.nii'
img = nib.load(nifti_file)
data = img.get_fdata()
num_slices = data.shape[2]  

# 初始化三维二值掩模
binary_volume = np.zeros_like(data, dtype=bool)

# 初始化中心线三维二值掩模
central_line_volume = np.zeros_like(data, dtype=bool)

# 找到最大圆
circles_info, binary_volume = search_circles(data, binary_volume)

# 打印所有区域的圆信息
for info in circles_info:
    slice_idx = info['slice']
    region_label = info['region_label']
    max_c = info['max_circle']
    sec_c = info['second_circle']
    thi_c = info['third_circle']
    print(f"Slice {slice_idx} - Region {region_label}:")
    print(f"  最大圆 - 圆心: {max_c['center']}, 半径: {max_c['radius']:.2f}")
    print(f"  第二个圆 - 圆心: {sec_c['center']}, 半径: {sec_c['radius']:.2f}")
    print(f"  第三个圆 - 圆心: {thi_c['center']}, 半径: {thi_c['radius']:.2f}")

# 允许的中心线两层之间最大距离
point_max_dist = 5

# 找中心线
central_lines = find_lines(circles_info, point_max_dist)

# 处理中心线，选择最长的四条并进行平滑
#central_smooth_line = process_central_lines(central_lines, window_size=9, top_n=4)
central_smooth_line = process_b_spline_central_lines(central_lines, smooth_factor=200, top_n=4)

# 打印结果验证
for line_name, smooth_line in central_smooth_line.items():
    print(f"{line_name}:")
    for point in smooth_line:
        print(f"  Layer {point['layer']}: Point {point['point']}")
    print("\n")

# 调用可视化函数
# visualize_lines(central_lines, central_smooth_line, num_samples=10)

# 将平滑后的中心线点转换为3D坐标，并在 central_line_volume 中设置相应的 voxels
for line_name, points in central_smooth_line.items():
    for point in points:
        y, x = point['point']
        z = point['layer']
        # 转换为整数索引
        y_int = int(round(y))
        x_int = int(round(x))
        z_int = int(z)
        # 边界检查
        if (0 <= y_int < data.shape[0] and
            0 <= x_int < data.shape[1] and
            0 <= z_int < data.shape[2]):
            central_line_volume[y_int, x_int, z_int] = True
        else:
            print(f"警告: 点 ({y_int}, {x_int}, {z_int}) 超出范围，未被添加到 central_line_volume。")

# 增强中心线的可视化（膨胀操作）
central_line_volume = morphology.binary_dilation(central_line_volume, morphology.ball(1))

# 生成三维表面网格
print("生成三维表面网格")
verts, faces, normals, values = measure.marching_cubes(binary_volume, level=0.5)
mesh_model = o3d.geometry.TriangleMesh()
mesh_model.vertices = o3d.utility.Vector3dVector(verts)
mesh_model.triangles = o3d.utility.Vector3iVector(faces)
mesh_model.compute_vertex_normals()
mesh_model.paint_uniform_color([0.7, 0.7, 0.7])  # 灰色
print("三维表面网格生成完毕.")

# 生成中心线的三维表面网格
print("生成中心线的三维表面网格")
verts_line, faces_line, normals_line, values_line = measure.marching_cubes(central_line_volume, level=0.5)
mesh_lines = o3d.geometry.TriangleMesh()
mesh_lines.vertices = o3d.utility.Vector3dVector(verts_line)
mesh_lines.triangles = o3d.utility.Vector3iVector(faces_line)
mesh_lines.compute_vertex_normals()
mesh_lines.paint_uniform_color([1, 0, 0])  # 红色
print("中心线的三维表面网格生成完毕.")

geometries = [mesh_model, mesh_lines]

# 可视化
print("可视化三维模型和中心线")
o3d.visualization.draw_geometries(
    geometries,
    window_name='3D Model with Central Lines',
    width=800,
    height=600,
    left=50,
    top=50,
    point_show_normal=False
)


Slice 0: 未检测到根管区域。
Slice 1: 未检测到根管区域。
Slice 2: 未检测到根管区域。
Slice 3: 未检测到根管区域。
Slice 4: 未检测到根管区域。
Slice 5: 未检测到根管区域。
Slice 6: 未检测到根管区域。
Slice 7: 未检测到根管区域。
Slice 8: 未检测到根管区域。
Slice 9: 未检测到根管区域。
Slice 10: 未检测到根管区域。
Slice 11: 未检测到根管区域。
Slice 12: 未检测到根管区域。
Slice 13: 未检测到根管区域。
Slice 14: 未检测到根管区域。
Slice 15: 未检测到根管区域。
Slice 16: 未检测到根管区域。
Slice 17: 未检测到根管区域。
Slice 582: 未检测到根管区域。
Slice 583: 未检测到根管区域。
Slice 584: 未检测到根管区域。
Slice 585: 未检测到根管区域。
Slice 586: 未检测到根管区域。
Slice 587: 未检测到根管区域。
Slice 588: 未检测到根管区域。
Slice 589: 未检测到根管区域。
Slice 590: 未检测到根管区域。
Slice 591: 未检测到根管区域。
Slice 592: 未检测到根管区域。
Slice 593: 未检测到根管区域。
Slice 594: 未检测到根管区域。
Slice 595: 未检测到根管区域。
Slice 596: 未检测到根管区域。
Slice 597: 未检测到根管区域。
Slice 598: 未检测到根管区域。
Slice 599: 未检测到根管区域。
Slice 600: 未检测到根管区域。
Slice 601: 未检测到根管区域。
Slice 602: 未检测到根管区域。
Slice 603: 未检测到根管区域。
Slice 604: 未检测到根管区域。
Slice 605: 未检测到根管区域。
Slice 606: 未检测到根管区域。
Slice 607: 未检测到根管区域。
Slice 608: 未检测到根管区域。
Slice 609: 未检测到根管区域。
Slice 610: 未检测到根管区域。
Slice 611: 未检测到根管区域。
Slice 612: 未检测到根管区域。

2024-10-20 18:03:28.998 python[60679:8640533] +[IMKClient subclass]: chose IMKClient_Legacy
2024-10-20 18:03:28.998 python[60679:8640533] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


In [2]:
import nibabel as nib
import numpy as np
from functions import search_circles, find_lines, process_central_lines, process_b_spline_central_lines, visualize_lines
from skimage import morphology, measure
import open3d as o3d
import matplotlib.pyplot as plt  
import matplotlib.cm as cm  

# 文件路径
#nifti_file = 'Cshape/5475-4_Canal.nii' 
nifti_file = 'qulv/5530-3/5530-3ori_canal.nii'
#nifti_file = 'all_c_shape/canal-before.nii'
img = nib.load(nifti_file)
data = img.get_fdata()
num_slices = data.shape[2]  

# 初始化三维二值掩模
binary_volume = np.zeros_like(data, dtype=bool)

# 初始化中心线三维二值掩模
central_line_volume = np.zeros_like(data, dtype=bool)

# 找到最大圆
circles_info, binary_volume = search_circles(data, binary_volume)

# 打印所有区域的圆信息
for info in circles_info:
    slice_idx = info['slice']
    region_label = info['region_label']
    max_c = info['max_circle']
    sec_c = info['second_circle']
    thi_c = info['third_circle']
    print(f"Slice {slice_idx} - Region {region_label}:")
    print(f"  最大圆 - 圆心: {max_c['center']}, 半径: {max_c['radius']:.2f}")
    print(f"  第二个圆 - 圆心: {sec_c['center']}, 半径: {sec_c['radius']:.2f}")
    print(f"  第三个圆 - 圆心: {thi_c['center']}, 半径: {thi_c['radius']:.2f}")

# 允许的中心线两层之间最大距离
point_max_dist = 5

# 找中心线
central_lines = find_lines(circles_info, point_max_dist)

# 处理中心线，选择最长的四条并进行平滑
#central_smooth_line = process_central_lines(central_lines, window_size=9, top_n=4)
central_smooth_line = process_b_spline_central_lines(central_lines, smooth_factor=200, top_n=4)

# 打印结果验证
for line_name, smooth_line in central_smooth_line.items():
    print(f"{line_name}:")
    for point in smooth_line:
        print(f"  Layer {point['layer']}: Point {point['point']}")
    print("\n")

# 计算曲率
def compute_curvature(points):
    """
    计算一条曲线的曲率。
    :param points: 曲线上的点列表，每个点为 [y, x, z]
    :return: 曲率列表，与点数量相同
    """
    curvatures = []
    for i in range(1, len(points)-1):
        p_prev = np.array(points[i-1])
        p_curr = np.array(points[i])
        p_next = np.array(points[i+1])

        v1 = p_curr - p_prev
        v2 = p_next - p_curr

        norm_v1 = np.linalg.norm(v1)
        norm_v2 = np.linalg.norm(v2)

        if norm_v1 == 0 or norm_v2 == 0:
            curvature = 0
        else:
            cos_theta = np.dot(v1, v2) / (norm_v1 * norm_v2)
            cos_theta = np.clip(cos_theta, -1.0, 1.0)  
            theta = np.arccos(cos_theta)
            curvature = theta / norm_v1  

        curvatures.append(curvature)

    # 对于首尾点，使用邻近点的曲率
    if len(curvatures) > 0:
        curvatures = [curvatures[0]] + curvatures + [curvatures[-1]]
    else:
        curvatures = [0] * len(points)

    return curvatures

# 映射曲率到颜色
def map_curvature_to_color(curvatures, colormap_name='jet'):
    """
    将曲率值映射到颜色。
    :param curvatures: 曲率列表
    :param colormap_name: 使用的matplotlib色图
    :return: 颜色列表，每个颜色为 [R, G, B]
    """
    cmap = cm.get_cmap(colormap_name)
    norm = plt.Normalize(vmin=min(curvatures), vmax=max(curvatures))
    colors = cmap(norm(curvatures))[:, :3]  # 去除alpha通道
    return colors.tolist()

# 创建带有曲率颜色的线集
curvature_line_sets = []
for line_name, smooth_line in central_smooth_line.items():
    points = []
    for point in smooth_line:
        y, x = point['point']
        z = point['layer']
        # 将索引转换为实际坐标，根据需要调整比例
        points.append([y+30, x, z])  

    curvatures = compute_curvature(points)
    colors = map_curvature_to_color(curvatures)

    # 创建线段（每两个相邻点之间一条线）
    lines = [[i, i + 1] for i in range(len(points) - 1)]
    # 对应每条线段分配一个颜色，取线段起点的颜色
    line_colors = [colors[i] for i in range(len(lines))]

    # 创建 Open3D 的 LineSet
    line_set = o3d.geometry.LineSet()
    line_set.points = o3d.utility.Vector3dVector(points)
    line_set.lines = o3d.utility.Vector2iVector(lines)
    line_set.colors = o3d.utility.Vector3dVector(line_colors)

    curvature_line_sets.append(line_set)

# 调用可视化函数
# visualize_lines(central_lines, central_smooth_line, num_samples=10)

# 将平滑后的中心线点转换为3D坐标，并在 central_line_volume 中设置相应的 voxels
for line_name, points in central_smooth_line.items():
    for point in points:
        y, x = point['point']
        z = point['layer']
        # 转换为整数索引
        try:
            y_int = int(round(y))
            x_int = int(round(x))
            z_int = int(z)
        except:
            x_int = -1
            y_int = -1
            z_int = -1
        # 边界检查
        if (0 <= y_int < data.shape[0] and
            0 <= x_int < data.shape[1] and
            0 <= z_int < data.shape[2]):
            central_line_volume[y_int, x_int, z_int] = True
        else:
            print(f"警告: 点 ({y_int}, {x_int}, {z_int}) 超出范围，未被添加到 central_line_volume。")

# 增强中心线的可视化（膨胀操作）
central_line_volume = morphology.binary_dilation(central_line_volume, morphology.ball(1))

# 生成三维表面网格
print("生成三维表面网格")
verts, faces, normals, values = measure.marching_cubes(binary_volume, level=0.5)
mesh_model = o3d.geometry.TriangleMesh()
mesh_model.vertices = o3d.utility.Vector3dVector(verts)
mesh_model.triangles = o3d.utility.Vector3iVector(faces)
mesh_model.compute_vertex_normals()
mesh_model.paint_uniform_color([0.7, 0.7, 0.7])  
print("三维表面网格生成完毕.")

# 生成中心线的三维表面网格
print("生成中心线的三维表面网格")
verts_line, faces_line, normals_line, values_line = measure.marching_cubes(central_line_volume, level=0.5)
mesh_lines = o3d.geometry.TriangleMesh()
mesh_lines.vertices = o3d.utility.Vector3dVector(verts_line)
mesh_lines.triangles = o3d.utility.Vector3iVector(faces_line)
mesh_lines.compute_vertex_normals()
mesh_lines.paint_uniform_color([1, 0, 0])  #红色
print("中心线的三维表面网格生成完毕.")

geometries = [mesh_model, mesh_lines]
#geometries = [mesh_model]

# 新增部分：添加带有曲率颜色的线集到几何体列表
geometries.extend(curvature_line_sets)

# 可视化
print("可视化三维模型和中心线")
o3d.visualization.draw_geometries(
    geometries,
    window_name='3D Model with Central Lines and Curvature',
    width=800,
    height=600,
    left=50,
    top=50,
    point_show_normal=False
)


Slice 0: 未检测到根管区域。
Slice 1: 未检测到根管区域。
Slice 2: 未检测到根管区域。
Slice 3: 未检测到根管区域。
Slice 4: 未检测到根管区域。
Slice 5: 未检测到根管区域。
Slice 6: 未检测到根管区域。
Slice 7: 未检测到根管区域。
Slice 8: 未检测到根管区域。
Slice 9: 未检测到根管区域。
Slice 10: 未检测到根管区域。
Slice 11: 未检测到根管区域。
Slice 12: 未检测到根管区域。
Slice 13: 未检测到根管区域。
Slice 14: 未检测到根管区域。
Slice 15: 未检测到根管区域。
Slice 16: 未检测到根管区域。
Slice 17: 未检测到根管区域。
Slice 582: 未检测到根管区域。
Slice 583: 未检测到根管区域。
Slice 584: 未检测到根管区域。
Slice 585: 未检测到根管区域。
Slice 586: 未检测到根管区域。
Slice 587: 未检测到根管区域。
Slice 588: 未检测到根管区域。
Slice 589: 未检测到根管区域。
Slice 590: 未检测到根管区域。
Slice 591: 未检测到根管区域。
Slice 592: 未检测到根管区域。
Slice 593: 未检测到根管区域。
Slice 594: 未检测到根管区域。
Slice 595: 未检测到根管区域。
Slice 596: 未检测到根管区域。
Slice 597: 未检测到根管区域。
Slice 598: 未检测到根管区域。
Slice 599: 未检测到根管区域。
Slice 600: 未检测到根管区域。
Slice 601: 未检测到根管区域。
Slice 602: 未检测到根管区域。
Slice 603: 未检测到根管区域。
Slice 604: 未检测到根管区域。
Slice 605: 未检测到根管区域。
Slice 606: 未检测到根管区域。
Slice 607: 未检测到根管区域。
Slice 608: 未检测到根管区域。
Slice 609: 未检测到根管区域。
Slice 610: 未检测到根管区域。
Slice 611: 未检测到根管区域。
Slice 612: 未检测到根管区域。

/var/folders/vh/pjl542w16gz9d4ftq8k0gxmh0000gn/T/ipykernel_67631/1696304141.py:100: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap(colormap_name)


生成三维表面网格
三维表面网格生成完毕.
生成中心线的三维表面网格
中心线的三维表面网格生成完毕.
可视化三维模型和中心线


2024-10-22 19:36:17.899 python[67631:9000071] +[IMKClient subclass]: chose IMKClient_Legacy
2024-10-22 19:36:17.899 python[67631:9000071] +[IMKInputSession subclass]: chose IMKInputSession_Legacy
